**Part 1:** EDA & Checking Model Assumptions


### Imports and Loading

In [1]:
#Imports 
# packages for numerics and data manipulation
import pandas as pd 
import numpy as np

# Packages for visualization
import matplotlib.pyplot as plt
import seaborn as sns


In [2]:
# loading the dataset 
taxi_df = pd.read_csv("2017_Yellow_Taxi_Trip_Data.csv")

In [3]:
# creating a copy of taxi_df where changes will go 
# can revert to 'taxi_df' if needed down the line
df = taxi_df.copy()

# display the dataset shape 
print(df.shape)

#display basic info about the datset
df.info()


(22699, 18)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 22699 entries, 0 to 22698
Data columns (total 18 columns):
 #   Column                 Non-Null Count  Dtype  
---  ------                 --------------  -----  
 0   Unnamed: 0             22699 non-null  int64  
 1   VendorID               22699 non-null  int64  
 2   tpep_pickup_datetime   22699 non-null  object 
 3   tpep_dropoff_datetime  22699 non-null  object 
 4   passenger_count        22699 non-null  int64  
 5   trip_distance          22699 non-null  float64
 6   RatecodeID             22699 non-null  int64  
 7   store_and_fwd_flag     22699 non-null  object 
 8   PULocationID           22699 non-null  int64  
 9   DOLocationID           22699 non-null  int64  
 10  payment_type           22699 non-null  int64  
 11  fare_amount            22699 non-null  float64
 12  extra                  22699 non-null  float64
 13  mta_tax                22699 non-null  float64
 14  tip_amount             22699 non-null  flo

checking for duplicates and missing values 

In [5]:
# check for missing values using .isna() and .drop_duplicates()

# Check for duplicates 
print("Shape of dataframe:", df.shape)
print("Shape of dataframe with duplicates dropped:", df.drop_duplicates().shape)

#check for missing values in dataframe
print('Total count of missing values:', df.isna().sum().sum())

#Display missing values per column in dataframe
print("Missing values per column:")
df.isna().sum()

Shape of dataframe: (22699, 18)
Shape of dataframe with duplicates dropped: (22699, 18)
Total count of missing values: 0
Missing values per column:


Unnamed: 0               0
VendorID                 0
tpep_pickup_datetime     0
tpep_dropoff_datetime    0
passenger_count          0
trip_distance            0
RatecodeID               0
store_and_fwd_flag       0
PULocationID             0
DOLocationID             0
payment_type             0
fare_amount              0
extra                    0
mta_tax                  0
tip_amount               0
tolls_amount             0
improvement_surcharge    0
total_amount             0
dtype: int64

There are no missing values in  the data 